In [2]:
import jax 
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

In [30]:
def lin_reg(theta,x):
    w, b = theta
    return w*x+b
def least_square(theta, x, y):
    pred = lin_reg(theta, x)
    return jnp.mean((pred - y)**2)
def update(theta, x, y, lr = 0.2):
    return theta - lr*jax.grad(least_square, argnums=(0))(theta, x, y)

In [36]:
x = jnp.linspace(0,1,101)
theta = jnp.array([1.,1.])
y = 3*jnp.linspace(0,1,101)+0.5
for i in range(200):
    theta = update(theta, x, y)

In [37]:
jax.grad(least_square)(theta, x, y)

DeviceArray([-0.00102905,  0.00055146], dtype=float32)